In [1]:
import pandas as pd

file_path = 'Dane-czyste.xlsx' 
df = pd.read_excel(file_path)

df.head()

,Package_Name,Destination,Region,Country,Category,Latitude,Longitude,Approximate_Annual_Tourists,Currency,Majority_Religion,...,All_Inclusive,Accommodation_Type,Transport_Type,Departure_City,Season,Group_Size,Trip_Length_Days,Tour_Popularity,Price_PLN,Price_per_day_PLN
0,Family in Rome,Rome,Lazio,Italy,City,41.902782,12.496366,14000000,Euro (EUR),Roman Catholic,...,excluded,Apartment,Flight,Wroclaw,Winter,26,14,4.0,5600,400
1,City Break in Florence,Florence,Tuscany,Italy,City,43.769581,11.255772,10000000,Euro (EUR),Roman Catholic,...,excluded,Hostel,Train,Poznan,Summer,15,9,5.0,2700,300
2,Cruise in Venice,Venice,Veneto,Italy,City,45.435559,12.336196,10000000,Euro (EUR),Roman Catholic,...,excluded,Apartment,Flight,Poznan,Spring,27,5,4.7,1300,260
3,Nature in Milan,Milan,Lombardy,Italy,City,45.464643,9.188540,7000000,Euro (EUR),Roman Catholic,...,excluded,Camping,Train,Poznan,Autumn,5,8,4.0,4800,600
4,Family in Naples,Naples,Campania,Italy,City,40.851330,14.254720,5000000,Euro (EUR),Roman Catholic,...,excluded,Hotel,Flight,Poznan,Summer,18,13,4.8,3120,240


In [2]:
#!pip install jupyter-dash
import pandas as pd
from dash import Dash, html, dash_table, dcc, Input, Output, html
from jupyter_dash import JupyterDash
import plotly.express as px
import base64
import plotly.graph_objects as go


MAPA Z WYBOREM PINEZKI, ABY ZOBACZYĆ SZCZEGÓŁY WYCIECZKI:

In [4]:
df["hover_info"] = (
    "<b>" + df["Package_Name"] + "</b><br>" +
    "Description: " + df["Cultural_Significance"] + "<br>" +
    "Destination: " + df["Destination"] + "<br>" +
    "Price: " + df["Price_PLN"].astype(str) + " PLN<br>" +
    "Days: " + df["Trip_Length_Days"].astype(str) + "<br>" +
    "All Inclusive: " + df["All_Inclusive"].astype(str)
)

with open("pinezka2.png", "rb") as image_file:
    encoded_pin = base64.b64encode(image_file.read()).decode()
image_url = "data:image/png;base64," + encoded_pin

fig = go.Figure()

fig.add_trace(go.Scattermapbox(
    lat = df["Latitude"],
    lon = df["Longitude"],
    mode = "markers",
    marker = dict(size = 25, color = "rgba(0,0,0,0)"),
    customdata = df["hover_info"],
    hovertemplate = "%{customdata}<extra></extra>"
))

pin_width = 0.15
pin_height = 0.15

fig.update_layout(
    mapbox = dict(
        style = "carto-positron",
        zoom = 3.5,
        center = dict(lat = df["Latitude"].mean(), lon = df["Longitude"].mean()),
        layers = [
            dict(
                sourcetype="image",
                source=image_url,
                coordinates=[
                    [lon - pin_width, lat + pin_height],
                    [lon + pin_width, lat + pin_height],
                    [lon + pin_width, lat - pin_height],
                    [lon - pin_width, lat - pin_height],
                ]
            )
            for lon, lat in zip(df["Longitude"], df["Latitude"])
        ]
    ),
    margin={"r": 0, "t": 0, "l": 0, "b": 0}
)

app = JupyterDash(__name__)
app.layout = html.Div([
    html.H2("Interactive European Tour Packages Map:"),
    dcc.Graph(id = 'map', figure = fig),
    html.Div(id = 'details', style = {"marginTop": "20px"})
])

@app.callback(
    Output('details', 'children'),
    Input('map', 'clickData')
)

def display_details(clickData):
    if clickData is None:
        return html.P("Click a marker to see trip details.")

    point_index = clickData['points'][0]['pointIndex']
    row = df.iloc[point_index]

    excluded = ["hover_info", "Latitude", "Longitude"]

    ordered_fields = (
        ["Package_Name", "Destination", "Region", "Country", "Cultural_Significance"]
        + [col for col in df.columns if col not in ["Package_Name", "Destination", "Region", "Country", "Cultural_Significance", "hover_info", "Latitude", "Longitude"]]
    )

    def prettify(name):
        if name == "Cultural_Significance":
            return "Description"
        return name.replace("_", " ").title()

    return html.Div([
        html.H3(row["Package_Name"], style = {"textAlign": "center", "marginBottom": "30px", "fontSize": "28px"}),
        html.Div([
            html.Div([
                html.P(prettify(col) + ":", style = {"fontWeight": "bold", "margin": "4px 0"}),
                html.P(str(row[col]), style = {"margin": "4px 0"})
            ]) for col in ordered_fields if col in row and col not in excluded
        ], style = {"columnCount": 2, "gap": "40px"})
    ], style = {
        "padding": "20px",
        "backgroundColor": "#f9f9f9",
        "border": "1px solid lightgray",
        "borderRadius": "8px"
    })

app.run(mode = 'inline')

WYBÓR KRAJU I INNYCH SZCZEGÓŁÓW, ABY ZOBACZYĆ CENY:

In [5]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H2("Trip Cost Analysis by Country", style = { "textAlign": "center", "marginBottom": "30px" }),

    html.Div([
        html.Label("Select a country:"),
        dcc.Dropdown(
            id = "country-dropdown",
            options = [ { "label": c, "value": c } for c in sorted(df["Country"].unique()) ],
            value = "Italy",
            clearable = False,
            style = { "width": "300px", "marginBottom": "10px" }
        ),

        dcc.Checklist(
            id = "all-inclusive-check",
            options = [ { "label": "Only All Inclusive", "value": "yes" } ],
            value = [],
            style = { "marginBottom": "20px" }
        ),

        html.Label("Select category:"),
        dcc.Dropdown(
            id = "xaxis-choice",
            options = [
                { "label": "Trip Length (days)", "value": "Trip_Length_Days" },
                { "label": "Trip Type", "value": "Trip_Type" },
                { "label": "Destination", "value": "Destination" }
            ],
            value = "Trip_Length_Days",
            clearable = False,
            style = { "width": "300px", "marginBottom": "20px" }
        )
    ], style = { "marginLeft": "20px" }),

    dcc.Graph(id = "scatter-cost-duration")
])

@app.callback(
    Output("scatter-cost-duration", "figure"),
    [ Input("country-dropdown", "value"),
      Input("all-inclusive-check", "value"),
      Input("xaxis-choice", "value") ]
)

def update_dynamic_bar(selected_country, all_inclusive_filter, x_column):
    filtered_df = df[ df["Country"] == selected_country ]

    if "yes" in all_inclusive_filter:
        filtered_df = filtered_df[ filtered_df["All_Inclusive"].str.lower() == "included" ]

    if filtered_df.empty:
        return go.Figure().update_layout(
            title = "No data available for selected filters",
            xaxis = { "visible": False },
            yaxis = { "visible": False }
        )

    grouped = filtered_df.groupby(x_column).agg({
        "Price_per_day_PLN": "mean",
        "Price_PLN": "mean",
        "Tour_Popularity": "mean"
    }).reset_index()

    grouped = grouped.sort_values("Price_per_day_PLN", ascending = True)

    fig = go.Figure()
    fig.add_trace(go.Bar(
        x = grouped[x_column],
        y = grouped["Price_per_day_PLN"],
        marker = dict(
            color = grouped["Tour_Popularity"],
            colorscale = "Blues",
            colorbar = dict(title = "Popularity")
        ),
        hovertemplate = (
            f"{x_column.replace('_', ' ')}: %{{x}}<br>"
            "Avg daily price: %{y:.0f} PLN<br>"
            "Avg total price: %{customdata[0]:.0f} PLN<br>"
            "Popularity: %{marker.color:.1f}<extra></extra>"
        ),
        customdata = grouped[["Price_PLN"]]
    ))

    fig.update_layout(
        title = f"Average Daily Price by {x_column.replace('_', ' ')} in{selected_country}:",
        xaxis_title = x_column.replace("_", " "),
        yaxis_title = "Average Price per Day (PLN)",
        height = 500,
        margin = dict(t = 50, l = 60, r = 60, b = 50)
    )

    return fig

app.run(mode = "inline")